In [ ]:
# !pip install transformers datasets huggingface_hub tensorboard==2.11
# !sudo apt-get install git-lfs --yes

## Instructions :


Change FILE_TO_PREDICT to your unanottated file name


Change ANNOTATED_FILE to your anottated file name for training : 

- sheets should contain at least columns ad_creative_body and 'Final label'. If number of labels is not 3 : Modify '#Modify here if more than 3 labels'

Modify NAME_FOR_YOUR_OUTPUT with the output name of the experiment

Modify keys with the sheets you want to use from the ANNOTATED_FILE

Modify '#Modify here with the name of the labels' : replace misinformation, cannot say, not misinformation with the actual names of your labels

In [ ]:
import torch
import os
import shutil
from torch import nn
from transformers import Trainer
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
    AutoTokenizer,
    AutoModel,
    BertForSequenceClassification
)
from huggingface_hub import HfFolder, notebook_login
from datasets import Dataset
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

from sklearn.metrics import classification_report

import json

In [ ]:
import pandas as pd
import json

table=pd.read_excel('ANNOTATED_FILE.xlsx', sheet_name=None)
for key in table:
    table[key]['sample_weights']=1.0

table_2=pd.read_excel('FILE_TO_PREDICT.xlsx')

In [ ]:
def get_percentages(data_list):
    ret=[]
    label_counts = {}
    total_items = len(data_list)
    
    for item in data_list:
        label = item["label"]
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1

    label_percentages = {}
    for label, count in label_counts.items():
        percentage = (count / total_items) * 100
        label_percentages[label] = percentage
    #Modify here if more than 3 labels
    ret=[label_percentages[0],label_percentages[1]]
    if 2 in label_percentages.keys():
        ret+=[label_percentages[2]]
    return ret
    


In [ ]:

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(weights, device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from torch import nn, tensor
import torch
# with sample weight and class weight. That's our final technic
class CustomTrainerWE(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # print(inputs.keys())
        labels = inputs.pop("labels")
        te=inputs["input_ids"]
        sample_weights_l=[]
        for i_te in range(len(te)):
            sample_weights_l += [dico_txt_to_weight[str([int(x) for x in te[i_te][0:10]])]]#torch.tensor(1)#inputs.pop("sample_weights")
        sample_weights=torch.tensor(sample_weights_l, device='cuda:0')
        # inputs["input_ids"]=te
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(weights, device=model.device), reduction='none')
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        loss = loss * sample_weights  
        loss = loss.mean()

        return (loss, outputs) if return_outputs else loss


In [ ]:

keys=['List','of','your','sheets in the xlsx you want to use for training']
nameexp='NAME_FOR_YOUR_OUTPUT'

bigdf= pd.concat([table[x] for x in keys], ignore_index=True)
bigdf=bigdf.sample(frac=1).reset_index(drop=True)
res_f_nf=[]
dico_txt_to_weight=dict()
res_f_nf_test=[]


for i in range(len(bigdf)):
  answers=bigdf.iloc[i]
  if not type(answers['ad_creative_body'])==str:
    continue
  text=answers['ad_creative_body']
  if not type(answers['Final label'])==str:
    continue
  finallabel=answers['Final label'].strip().lower() 
  answer={'cannot say':1, 'misinformation':2, 'not misinformation':0}[finallabel] #Modify here with the name of the labels
  sample_weights=answers['sample_weights']
  
  res_f_nf+=[{'text':text, 'label':answer,"sample_weights":sample_weights}]


for i in range(len(table_2)):
  answers=table_2.iloc[i]
  if not type(answers['ad_creative_body'])==str:
    continue
  text=answers['ad_creative_body']
  id_r=answers['id']
  res_f_nf_test+=[{'text':text, 'id':id_r}]

dsnf_tr=Dataset.from_pandas(pd.DataFrame(data=res_f_nf))
# dsnf_test=Dataset.from_pandas(pd.DataFrame(data=res_f_nf_test))
    


# f=open(nameexp+'_train.json','w')
# json.dump(res_f_nf[:-testsize],f)
# f.close()

# f=open(nameexp+'_test.json','w')
# json.dump(res_f_nf_test[-testsize:],f)
# f.close()
model_id = "digitalepidemiologylab/covid-twitter-bert-v2"# "Darkrider/covidbert_medmarco"
outmodel_dir=nameexp+'_out'




# Load dataset
#dataset = load_dataset(dataset_id)
# train_dataset = dsnf_tr#dataset['train']
# test_dataset = split_datasets['test']# dsnf_test#.shard(num_shards=2, index=0)#dataset["test"].shard(num_shards=2, index=0)test_dataset = split_datasets['test']# dsnf_test#.shard(num_shards=2, index=0)#dataset["test"].shard(num_shards=2, index=0)




# Load dataset
# dataset = load_dataset(dataset_id)
train_dataset = dsnf_tr  # Your original training dataset
# print(train_dataset[0]) 
# Split the dataset into 90% train and 10% validation
split_datasets = train_dataset.train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
val_dataset = split_datasets['test']
test_dataset = split_datasets['test']# dsnf_test#.shard(num_shards=2, index=0)#dataset["test"].shard(num_shards=2, index=0)test_dataset = split_datasets['test']# dsnf_test#.shard(num_shards=2, index=0)#dataset["test"].shard(num_shards=2, index=0)
# Split train_dataset into train and validation sets
# val_dataset = dsnf_test.shard(num_shards=2, index=1)

# Preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_id)

# tokenizer =RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
# print(train_dataset[0]) 
# print(train_dataset.keys())
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label","sample_weights"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label","sample_weights"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label","sample_weights"])
# print(train_dataset[0]) 
# Extract the number of classess and their names
num_labels = 3#Modify here if more than 3 labels
class_names = [0,1,2]#Modify here if more than 3 labels
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this to directly output the class names when using the pipeline without needing to map the labels later.
id2label = {i: label for i, label in enumerate(class_names)}

# 3. Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

for elt in train_dataset:

    dico_txt_to_weight[str([int(x) for x in elt['input_ids'][0:10]])]=elt['sample_weights']
for elt in val_dataset:

    dico_txt_to_weight[str([int(x) for x in elt['input_ids'][0:10]])]=elt['sample_weights']




# model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)
model = BertForSequenceClassification.from_pretrained(model_id, config=config, ignore_mismatched_sizes=True)

# TrainingArguments
training_args = TrainingArguments(
    output_dir=outmodel_dir,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir=f"{outmodel_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    # report_to="tensorboard",
    # push_to_hub=True,
    # hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)


weights = get_percentages(res_f_nf)  
# sv_weights_or+=[weights]
weights=[1/x for x in weights]
total = sum(weights)
weights = [w/total for w in weights]


trainer = CustomTrainerWE(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



# Fine-tune the model
trainer.train()

trainer.evaluate()
model=model.to('cpu')



resfinal_f_nf=[]
totalnb=0
totalcorrect=0
gold={0:0,1:0,2:0}#Modify here if more than 3 labels'
pred={0:0,1:0,2:0}#Modify here if more than 3 labels'


texts=[elt['text'] for elt in res_f_nf_test]
# answers=[elt['label'] for elt in res_f_nf[-testsize:]]
pipe = pipeline('text-classification',tokenizer=tokenizer,model=model)#outmodel_dir+'/checkpoint-500')

predictions_fsci=[]
answers_fsci=[]

for i,elt in tqdm(enumerate(res_f_nf_test)):
  try:
    result = pipe(texts[i])


    if not elt['text']==texts[i]:
      print('problem')
    prediction=result[0]['label']
    elt['prediction']=prediction

    predictions_fsci+=[prediction]
    

    pred[prediction]+=1
    totalnb+=1
    resfinal_f_nf+=[elt]
    # print(result[0]['generated_text'])
  except Exception as e:

    print('error')

resfinal_f_nf+=[{'totalnb':totalnb,'pred':pred}]





import copy
import numpy
resfinal_f_nf_2=[]
for x in resfinal_f_nf:
    x2=copy.deepcopy(x)
    for elt in x2.keys():
        if type(x2[elt])==numpy.int64:
            x2[elt]=int(x2[elt])
        if type(x2[elt])==numpy.int64:
            print('?')
    resfinal_f_nf_2+=[x2]

f=open('results_'+nameexp+'.json','w')
json.dump(resfinal_f_nf_2,f, indent=4)
f.close()

folder_name = nameexp + '_out'

# Check if the folder exists
if os.path.exists(folder_name):
    # Remove the folder and all its contents
    shutil.rmtree(folder_name)
    print(f"Folder '{folder_name}' and all its subdirectories have been removed.")
else:
    print(f"Folder '{folder_name}' does not exist.")


In [ ]:
pdnew=pd.DataFrame(resfinal_f_nf_2)

In [ ]:
pdnew

In [ ]:
pdnew=pdnew.drop(['totalnb','pred'],axis=1)

In [ ]:
list(set(bigdf['Final label']))

In [ ]:
list(set(pdnew['prediction']))

In [ ]:
pdnew.to_excel('results_'+nameexp+'.xlsx')

#Here we have the results
#The following code only serves to check results for a specific label, in our case misinformation that was underrepresented

# Optional 

In [ ]:
# keys=['US ad', 'US_ad_v600', 'US_ad_nocovid_1000']

bigdf2= pd.concat([table[x] for x in keys], ignore_index=True)

In [ ]:
filtered_df=copy.deepcopy(bigdf2)

In [ ]:
filtered_df['Final label'] =filtered_df['Final label'].str.strip().str.lower()

In [ ]:
filtered_df = filtered_df[filtered_df['Final label'] =='misinformation']

In [ ]:
misinfo_claims=list(set(list(filtered_df['ad_creative_body'])))

In [ ]:

u=list(zip(misinfo_claims,[x['label'] for x in pipe(misinfo_claims)]))

In [ ]:
print('\n'.join([t[0].replace('\n','')+ ' : ' + str(t[1])+'\n' for t in u if t[1]==0]))

In [ ]:
####NEW
print('0')
print('\n'.join([t[0].replace('\n','')+ ' : ' + str(t[1])+'\n' for t in u if t[1]==0]))

print('1')
print('\n'.join([t[0].replace('\n','')+ ' : ' + str(t[1])+'\n' for t in u if t[1]==1]))

print('2')
print('\n'.join([t[0].replace('\n','')+ ' : ' + str(t[1])+'\n' for t in u if t[1]==2]))

In [ ]:
Counter([x['label'] for x in pipe( misinfo_claims)])